In [1]:
# Import findspark and initialize.
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [874 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:13 http://security.ubu

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [8]:
# Preview DataFrame
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header = True)
data_df.head(5)

[Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built='2016', price='936923', bedrooms='4', bathrooms='3', sqft_living='3167', sqft_lot='11733', floors='2', waterfront='1', view='76'),
 Row(id='7530a2d8-1ae3-4517-9f4a-befe060c4353', date='2021-06-13', date_built='2013', price='379628', bedrooms='2', bathrooms='2', sqft_living='2235', sqft_lot='14384', floors='1', waterfront='0', view='23'),
 Row(id='43de979c-0bf0-4c9f-85ef-96dc27b258d5', date='2019-04-12', date_built='2014', price='417866', bedrooms='2', bathrooms='2', sqft_living='2127', sqft_lot='10575', floors='2', waterfront='0', view='0'),
 Row(id='b672c137-b88c-48bf-9f18-d0a4ac62fb8b', date='2019-10-16', date_built='2016', price='239895', bedrooms='2', bathrooms='2', sqft_living='1631', sqft_lot='11149', floors='2', waterfront='0', view='0'),
 Row(id='e0726d4d-d595-4074-8283-4139a54d0d63', date='2022-01-08', date_built='2017', price='424418', bedrooms='3', bathrooms='2', sqft_living='2249', sqft_lot='13878

In [9]:
# 2. Create a temporary view of the DataFrame.
data_df.createOrReplaceTempView('home_sales')


In [16]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
four_bed_average = spark.sql("SELECT YEAR (date) AS year_sold, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms == 4 GROUP BY YEAR(date) ORDER BY YEAR(date)")
four_bed_average.show()

+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [18]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bed_bath_average = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY date_built ORDER BY date_built")
three_bed_bath_average.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [22]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
three_bed_bath_two_floor_average = spark.sql("SELECT date_built, ROUND(AVG(price), 2) as average_price FROM home_sales WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
three_bed_bath_two_floor_average.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [23]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_average = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM home_sales GROUP BY view HAVING ROUND(AVG(price), 2) >= 350000")
view_average.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  51|    788128.21|
|  54|    798684.82|
|  69|    750537.94|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|  59|     791453.0|
|  85|   1056336.74|
|  52|    733780.26|
|  71|     775651.1|
|  98|   1053739.33|
|  99|   1061201.42|
|  96|   1017815.92|
| 100|    1026669.5|
|  70|    695865.58|
|  61|    746877.59|
|  75|   1114042.94|
|  78|   1080649.37|
|  89|   1107839.15|
|  77|   1076205.56|
+----+-------------+
only showing top 20 rows

--- 0.8403444290161133 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [26]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

view_average = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM home_sales GROUP BY view HAVING ROUND(AVG(price), 2) >= 350000")
view_average.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  51|    788128.21|
|  54|    798684.82|
|  69|    750537.94|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|  59|     791453.0|
|  85|   1056336.74|
|  52|    733780.26|
|  71|     775651.1|
|  98|   1053739.33|
|  99|   1061201.42|
|  96|   1017815.92|
| 100|    1026669.5|
|  70|    695865.58|
|  61|    746877.59|
|  75|   1114042.94|
|  78|   1080649.37|
|  89|   1107839.15|
|  77|   1076205.56|
+----+-------------+
only showing top 20 rows

--- 0.4203155040740967 seconds ---


In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
parquet_df = data_df.write.partitionBy("date_built").mode("overwrite").parquet("home_parquet")

In [29]:
# 11. Read the formatted parquet data.
parquet_data = spark.read.parquet("home_parquet")

In [30]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView('temporary_parquet')

In [31]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

parq_average = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM temporary_parquet GROUP BY view HAVING ROUND(AVG(price), 2) >= 350000")
parq_average.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  51|    788128.21|
|  54|    798684.82|
|  69|    750537.94|
|  73|    752861.18|
|  87|    1072285.2|
|  64|    767036.67|
|  59|     791453.0|
|  85|   1056336.74|
|  52|    733780.26|
|  71|     775651.1|
|  98|   1053739.33|
|  99|   1061201.42|
|  96|   1017815.92|
| 100|    1026669.5|
|  70|    695865.58|
|  61|    746877.59|
|  75|   1114042.94|
|  78|   1080649.37|
|  89|   1107839.15|
|  77|   1076205.56|
+----+-------------+
only showing top 20 rows

--- 1.083150863647461 seconds ---


In [32]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [33]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False